# Bab 8 Backpropagation (2)

## Praktikum

### a) Fungsi *Training* Backpropagation

Tulis kode ke dalam *cell* di bawah ini:

In [23]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

In [16]:
def sig(X):
  return [1/(1 + np.exp(-x)) for x in X]

def sigd(X):
  output = []

  for i, x in enumerate(X):
    s = sig([x])[0]

    output.append(s * (1 - s))

  return output

In [17]:
def bp_fit(X, target, layer_conf, max_epoch, max_error = .1, learn_rate = .1, print_per_epoch=100):
    np.random.seed(1)
    nin = [np.empty(i) for i in layer_conf]
    n = [np.empty(j + 1) if i < len(layer_conf) - 1 else np.empty(j) for i, j in enumerate(layer_conf)]
    # w = np.array([np.random.rand(layer_conf[i] + 1, layer_conf[i + 1]) for i in range(len(layer_conf) - 1)])
    w = [np.random.rand(layer_conf[i] + 1, layer_conf[i + 1]) for i in range(len(layer_conf) - 1)]
    dw = [np.empty((layer_conf[i] + 1, layer_conf[i + 1])) for i in range(len(layer_conf) - 1)]
    d = [np.empty(s) for s in layer_conf[1:]]
    din = [np.empty(s) for s in layer_conf[1:-1]]
    epoch = 0
    mse = 1

    for i in range(0, len(n)-1):
      n[i][-1] = 1

      while (max_epoch == -1 or epoch < max_epoch) and mse > max_error:
        epoch += 1
        mse = 0

        for r in range(len(X)):
          n[0][:-1] = X[r]

          for L in range(1, len(layer_conf)):
            nin[L] = np.dot(n[L-1], w[L-1])
            n[L][:len(nin[L])] = sig(nin[L])

          e = target[r] - n[-1]
          mse += sum(e ** 2)
          d[-1] = e * sigd(nin[-1])
          dw[-1] = learn_rate * d[-1] * n[-2].reshape((-1, 1))

          for L in range(len(layer_conf) - 1, 1, -1):
            din[L-2] = np.dot(d[L-1], np.transpose(w[L-1][:-1]))
            d[L-2] = din[L-2] * np.array(sigd(nin[L-1]))
            dw[L-2] = (learn_rate * d[L-2]) * n[L-2].reshape((-1, 1))

          for i in range(len(w)):
            w[i] += dw[i]

        mse /= len(X)

        if print_per_epoch > -1 and epoch % print_per_epoch == 0:
          print(f'Epoch: {epoch}, MSE: {mse}')

      return w, epoch, mse

### b) Fungsi *Testing* Backpropagation

Tulis kode ke dalam *cell* di bawah ini:

In [18]:
def bp_predict(X, w):
  n = [np.empty(len(i)) for i in w]
  nin = [np.empty(len(i[0])) for i in w]
  predict = []

  n.append(np.empty(len(w[-1][0])))

  for x in X:
    n[0][:-1] = x

    for L in range(0, len(w)):
      nin[L] = np.dot(n[L], w[L])
      n[L+1][:len(nin[L])] = sig(nin[L])

    predict.append(n[-1].copy())

  return predict

### c) Percobaan Klasifikasi Dataset Iris

![Iris Dataset](https://www.spataru.at/images/blog/iris-dataset-svm/iris_types.jpg)

Tulis kode ke dalam *cell* di bawah ini:

In [19]:
def onehot_enc(lbl, min_val=0):
  mi = min(lbl)
  enc = np.full((len(lbl), max(lbl) - mi + 1), min_val, np.int8)

  for i, x in enumerate(lbl):
    enc[i, x - mi] = 1

  return enc

def onehot_dec(enc, mi=0):
  return [np.argmax(e) + mi for e in enc]

In [20]:
iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = onehot_enc(iris.target)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3, random_state=1)
w, ep, mse = bp_fit(X_train, y_train, layer_conf = (4,3,3), learn_rate = .1, max_epoch = 1000, max_error = .1, print_per_epoch = 25)

print(f'Epoch: {ep}, MSE: {mse}')

predict = bp_predict(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)

print('Output:', predict)
print('Target:', y_test)
print('Accuracy:', accuracy)

Epoch: 25, MSE: 0.3833166779443107
Epoch: 50, MSE: 0.301396185088815
Epoch: 75, MSE: 0.26148932074548786
Epoch: 100, MSE: 0.19740347109913006
Epoch: 125, MSE: 0.13591678154244755
Epoch: 150, MSE: 0.1005247355667754
Epoch: 151, MSE: 0.09953734654801856
Output: [0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 2, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 2, 2, 1]
Target: [0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 2, 2, 1]
Accuracy: 0.9777777777777777


Analisis
1. Lakukan klasifikasi dengan menggunakan dataset Iris seperti di atas.
Ubahlah beberapa pengaturan sebagai berikut:
* Rasio data latih 70% dan data uji 30%
* Hidden neuron = 2
* Max epoch = 100
* Learning rate = 0,1
* Max error = 0,5

 pengujian (testing) menggunakan data latih dan data uji. Bandingkan nilai akurasi yang didapatkan. Fenomena apa yang terjadi pada pengujian ini? Mengapa hal tersebut terjadi?

2. Lakukan klasifikasi dengan menggunakan dataset Iris seperti di atas.
Ubahlah beberapa pengaturan sebagai berikut:
* Rasio data latih 70% dan data uji 30%
* Hidden neuron = 25
* Max epoch = 10000
* Learning rate = 0,1
* Max error = 0,01

  Lakukan pengujian (testing) menggunakan data latih dan data uji. Bandingkan nilai akurasi yang didapatkan. Fenomena apa yang terjadi pada pengujian ini? Mengapa hal tersebut terjadi?

3. Ubahlah parameter berikut agar mendapatkan akurasi tertinggi saat
melakukan testing menggunakan data uji :
* Hidden neuron
* Max epoch
* Max error

  Berapakah nilai akurasi tertinggi yang dapat Anda peroleh? Berapakah nilai masing-masing parameter tersebut?



JAWABAN


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3, random_state=1)
w, ep, mse = bp_fit(X_train, y_train, layer_conf = (4,2,3), learn_rate = .1, max_epoch = 100, max_error = .5, print_per_epoch = 25)

print(f'Epoch: {ep}, MSE: {mse}')

predict1 = bp_predict(X_test, w)
predict1 = onehot_dec(predict1)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict1, y_test)

print('Output:', predict1)
print('Target:', y_test)
print('Accuracy:', accuracy)

Epoch: 25, MSE: 0.6023662849933146
Epoch: 33, MSE: 0.48792259684397216
Output: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Target: [0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 2, 2, 1]
Accuracy: 0.28888888888888886


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3, random_state=1)
w, ep, mse = bp_fit(X_train, y_train, layer_conf = (4,25,3), learn_rate = .1, max_epoch = 10000, max_error = .01, print_per_epoch = 25)

print(f'Epoch: {ep}, MSE: {mse}')

predict2 = bp_predict(X_test, w)
predict2 = onehot_dec(predict2)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict2, y_test)

print('Output:', predict2)
print('Target:', y_test)
print('Accuracy:', accuracy)

Epoch: 25, MSE: 1.9993202764736877
Epoch: 50, MSE: 1.9985480410601613
Epoch: 75, MSE: 1.558898734035213
Epoch: 100, MSE: 0.9724997905330577
Epoch: 125, MSE: 0.3168162131601929
Epoch: 150, MSE: 0.24245559937220204
Epoch: 175, MSE: 0.1854405746593738
Epoch: 200, MSE: 0.13987973956674984
Epoch: 225, MSE: 0.11042567912241297
Epoch: 250, MSE: 0.09151661694263978
Epoch: 275, MSE: 0.07920661111748672
Epoch: 300, MSE: 0.07097830670337435
Epoch: 325, MSE: 0.06525694176677267
Epoch: 350, MSE: 0.06110637536408314
Epoch: 375, MSE: 0.05797684045342432
Epoch: 400, MSE: 0.055538708436774935
Epoch: 425, MSE: 0.053587334791136684
Epoch: 450, MSE: 0.05199082749692931
Epoch: 475, MSE: 0.05066113680997586
Epoch: 500, MSE: 0.049537570783459786
Epoch: 525, MSE: 0.048577052520919414
Epoch: 550, MSE: 0.04774816176038993
Epoch: 575, MSE: 0.04702738309280835
Epoch: 600, MSE: 0.046396686886841225
Epoch: 625, MSE: 0.045841938648387134
Epoch: 650, MSE: 0.04535183474420115
Epoch: 675, MSE: 0.044917178008453414
Epoc

In [25]:
hidden_neurons_options = [3, 5, 7]
max_epoch_options = [500, 1000, 1500]
max_error_options = [0.1, 0.05, 0.01]

iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = onehot_enc(iris.target)

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.3, random_state=1)

best_accuracy = 0
best_params = None

for hidden_neurons in hidden_neurons_options:
    for max_epoch in max_epoch_options:
        for max_error in max_error_options:
            print(f"Training dengan hidden_neurons={hidden_neurons}, max_epoch={max_epoch}, max_error={max_error}")

            layer_conf = (4, hidden_neurons, 3)
            w, ep, mse = bp_fit(X_train, y_train, layer_conf=layer_conf, max_epoch=max_epoch, max_error=max_error)

            predictions = bp_predict(X_val, w)
            predictions = onehot_dec(predictions)
            y_val_decoded = onehot_dec(y_val)
            accuracy = accuracy_score(predictions, y_val_decoded)

            print(f"Akurasi validasi: {accuracy}")

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = {'hidden_neurons': hidden_neurons, 'max_epoch': max_epoch, 'max_error': max_error}

print(f"Parameter Terbaik: {best_params}")
print(f"Akurasi validasi terbaik: {best_accuracy}")

Training dengan hidden_neurons=3, max_epoch=500, max_error=0.1
Epoch: 100, MSE: 0.19740347109913006
Akurasi validasi: 0.9777777777777777
Training dengan hidden_neurons=3, max_epoch=500, max_error=0.05
Epoch: 100, MSE: 0.19631456450669632
Epoch: 200, MSE: 0.0712322341576985
Epoch: 300, MSE: 0.05459792229901141
Akurasi validasi: 0.9777777777777777
Training dengan hidden_neurons=3, max_epoch=500, max_error=0.01
Epoch: 100, MSE: 0.19740347109913006
Epoch: 200, MSE: 0.07135124444502335
Epoch: 300, MSE: 0.05466114541687549
Epoch: 400, MSE: 0.04884497131588288
Epoch: 500, MSE: 0.04584978664862286
Akurasi validasi: 0.9777777777777777
Training dengan hidden_neurons=3, max_epoch=1000, max_error=0.1
Epoch: 100, MSE: 0.19631456450669632
Akurasi validasi: 0.9777777777777777
Training dengan hidden_neurons=3, max_epoch=1000, max_error=0.05
Epoch: 100, MSE: 0.19740347109913006
Epoch: 200, MSE: 0.07135124444502335
Epoch: 300, MSE: 0.05466114541687549
Akurasi validasi: 0.9777777777777777
Training dengan

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3, random_state=1)
w, ep, mse = bp_fit(X_train, y_train, layer_conf = (4,3,3), learn_rate = .1, max_epoch = 1000, max_error = .01, print_per_epoch = 25)

print(f'Epoch: {ep}, MSE: {mse}')

predict2 = bp_predict(X_test, w)
predict2 = onehot_dec(predict2)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict2, y_test)

print('Output:', predict2)
print('Target:', y_test)
print('Accuracy:', accuracy)

Epoch: 25, MSE: 0.38400588259178264
Epoch: 50, MSE: 0.3015244345783775
Epoch: 75, MSE: 0.2610489438635036
Epoch: 100, MSE: 0.19631456450669632
Epoch: 125, MSE: 0.13515114584104024
Epoch: 150, MSE: 0.10015545476521427
Epoch: 175, MSE: 0.08173256513095625
Epoch: 200, MSE: 0.0712322341576985
Epoch: 225, MSE: 0.06466815604267687
Epoch: 250, MSE: 0.060226671609326925
Epoch: 275, MSE: 0.057024195347387535
Epoch: 300, MSE: 0.05459792229901141
Epoch: 325, MSE: 0.052688748009564
Epoch: 350, MSE: 0.051142626862461106
Epoch: 375, MSE: 0.04986288408483368
Epoch: 400, MSE: 0.04878582540940057
Epoch: 425, MSE: 0.047867616003047175
Epoch: 450, MSE: 0.04707689616775968
Epoch: 475, MSE: 0.04639045462432012
Epoch: 500, MSE: 0.04579059931195386
Epoch: 525, MSE: 0.045263505206232416
Epoch: 550, MSE: 0.04479814272913038
Epoch: 575, MSE: 0.04438556127693989
Epoch: 600, MSE: 0.044018395866749195
Epoch: 625, MSE: 0.0436905176374531
Epoch: 650, MSE: 0.043396779501970684
Epoch: 675, MSE: 0.04313282637023105
Epo